# data.ipynb
AI HUB 데이터를 DropoutNet의 Input으로 변환하는 메소드를 제공하는 라이브러리

1. 국내 여행로그 데이터셋 전처리
2. traveler_area_rating_matrix 생성
3. User, Travel latent Vector 생성
4. user information vector 생성
5. train.csv, test.csv, test_item_ids.csv 생성


# CITEULike 데이터셋 상세 설명
- WRMF_U: (5551 + 1, 200), 첫 행은 0으로 구성
- WRMF_V: (16980 + 1, 200), 첫 행은 0으로 구성
- item_feature: 16981개 --> Item 개수와 동일
- train.csv: item id 기준 0.8 비율
- test.csv: item id 기준 0.2 비율
- test_item_ids.csv: test.csv의 iid 오름차순 정렬

item에 대한 user의 평점 행렬을
item에 대해 8:2로 train, test 비율을 나눔

1. 전체 rating_matrix 있음
2. item_id 추출 0.8, 0.2로 train/test 나눔 -> train_id, test_id
3. train/test 비율대로 rating_matrix 나눔 -> train_rating_matrix, test_rating_matrix

# AI HUB 데이터셋 (수도권 기준)
- WRMF_U: (12748 + 1, 200), 첫 행은 0으로 구성
- WRMF_V: (31472 + 1, 200), 첫 행은 0으로 구성
- user_feature: 12748 + 1개 --> Item 개수와 동일
- train.csv: item id 기준 0.8 비율
- test.csv: item id 기준 0.2 비율
- test_item_ids.csv: test.csv의 iid 오름차순 정렬

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 18.3 MB/s eta 0:00:00


In [3]:
from implicit.nearest_neighbours import bm25_weight
from implicit.cpu.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
import implicit
import datetime
import os

from geopy.geocoders import Nominatim # 위도 경도 -> 주소 변환

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [4]:
# 기본 디렉토리 설정
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/Capstone Design 2023 2nd/DropoutNet/Dataset/all_travel_log"

sudo_dir = os.path.join(dataset_dir + "/sudo")
east_dir = os.path.join(dataset_dir + "/east")
west_dir = os.path.join(dataset_dir + "/west")
jeju_dir = os.path.join(dataset_dir + "/jeju")

In [33]:
def concat_dataset(sudo_dir, east_dir, west_dir, jeju_dir):
  # 1. 데이터 경로 설정
  sudo_train_dir = os.path.join(sudo_dir + "/Training/raw/TS_csv")
  east_train_dir = os.path.join(east_dir + "/Training/raw/TS_csv")
  west_train_dir = os.path.join(west_dir + "/Training/raw/TS_csv")
  jeju_train_dir = os.path.join(jeju_dir + "/Training/raw/TS_csv")

  sudo_valid_dir = os.path.join(sudo_dir + "/Validation/raw/VS_csv")
  east_valid_dir = os.path.join(east_dir + "/Validation/raw/VS_csv")
  west_valid_dir = os.path.join(west_dir + "/Validation/raw/VS_csv")
  jeju_valid_dir = os.path.join(jeju_dir + "/Validation/raw/VS_csv")

  # 2. 데이터 불러오기
  # 2.1 수도권 데이터
  sudo_vr_travel_df = pd.read_csv(sudo_valid_dir + "/tn_travel_A.csv")
  sudo_tr_travel_df = pd.read_csv(sudo_train_dir + '/tn_travel_A.csv')
  # sudo_tr_area_df = pd.read_csv(sudo_train_dir + '/tn_visit_area_info_A.csv')
  # sudo_vr_area_df = pd.read_csv(sudo_valid_dir + "/tn_visit_area_info_A.csv")
  sudo_tr_traveller_df = pd.read_csv(sudo_train_dir + "/tn_traveller_master_A.csv")
  sudo_vr_traveller_df = pd.read_csv(sudo_valid_dir + "/tn_traveller_master_A.csv")

  # 2.1 동부권 데이터
  east_tr_travel_df = pd.read_csv(east_train_dir + '/tn_travel_B.csv')
  east_vr_travel_df = pd.read_csv(east_valid_dir + "/tn_travel_B.csv")
  # east_tr_area_df = pd.read_csv(east_train_dir + '/tn_visit_area_info_B.csv')
  # east_vr_area_df = pd.read_csv(east_valid_dir + "/tn_visit_area_info_B.csv")
  east_tr_traveller_df = pd.read_csv(east_train_dir + "/tn_traveller_master_B.csv")
  east_vr_traveller_df = pd.read_csv(east_valid_dir + "/tn_traveller_master_B.csv")

  # 2.1 서부권 데이터
  west_tr_travel_df = pd.read_csv(west_train_dir + '/tn_travel_C.csv')
  west_vr_travel_df = pd.read_csv(west_valid_dir + "/tn_travel_C.csv")
  # west_tr_area_df = pd.read_csv(west_train_dir + '/tn_visit_area_info_C.csv')
  # west_vr_area_df = pd.read_csv(west_valid_dir + "/tn_visit_area_info_C.csv")
  west_tr_traveller_df = pd.read_csv(west_train_dir + "/tn_traveller_master_C.csv")
  west_vr_traveller_df = pd.read_csv(west_valid_dir + "/tn_traveller_master_C.csv")

  # 2.1 제주권 데이터
  jeju_tr_travel_df = pd.read_csv(jeju_train_dir + '/tn_travel_D.csv')
  jeju_vr_travel_df = pd.read_csv(jeju_valid_dir + "/tn_travel_D.csv")
  # jeju_tr_area_df = pd.read_csv(jeju_train_dir + '/tn_visit_area_info_D.csv')
  # jeju_vr_area_df = pd.read_csv(jeju_valid_dir + "/tn_visit_area_info_D.csv")
  jeju_tr_traveller_df = pd.read_csv(jeju_train_dir + "/tn_traveller_master_D.csv")
  jeju_vr_traveller_df = pd.read_csv(jeju_valid_dir + "/tn_traveller_master_D.csv")

  # 2. 전체 concat
  concat_travel_df = pd.concat([sudo_tr_travel_df, sudo_vr_travel_df, east_tr_travel_df, east_vr_travel_df, west_tr_travel_df, west_vr_travel_df, jeju_tr_travel_df, jeju_vr_travel_df])
  # concat_area_df = pd.concat([sudo_tr_area_df, sudo_vr_area_df, east_tr_area_df, east_vr_area_df, west_tr_area_df, west_vr_area_df, jeju_tr_area_df, jeju_vr_area_df])
  concat_area_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone Design 2023 2nd/DropoutNet/Dataset/all.csv')
  concat_traveller_df = pd.concat([sudo_tr_traveller_df, sudo_vr_traveller_df, east_tr_traveller_df, east_vr_traveller_df, west_tr_traveller_df, west_vr_traveller_df, jeju_tr_traveller_df, jeju_vr_traveller_df])

  # 3. 결과 반환
  return concat_travel_df, concat_area_df, concat_traveller_df

In [18]:
# method to convert X_COORD, Y_COORD to Address
def convert_coordinate_to_address(X_COORD, Y_COORD):
  XY_COORD_str = f"{str(X_COORD)}, {str(Y_COORD)}"

  geolocoder = Nominatim(user_agent = "South Korea", timeout=None)
  address = geolocoder.reverse(XY_COORD_str, exactly_one=True)

  return address

In [7]:
# method to split address to SIDO address
def split_address_to_SIDO(address):
  pass


In [38]:
# method to create rating matrix
def create_rating_matrix(concat_merged_df):
  """
    Input
    concat_merged_df는 [TRAVEL_ID TRAVELER_ID  VISIT_AREA_ID  DGSTFN]와 같는 열을 지닌다.
    - concat_merged_df: train, valid의 데이터가 결합된 데이터 프레임

    Output
    - rating_matrix: 3개의 열["USER_ID", "ITEM_ID", "RATING"]로 구성된 Dataframe이다.
  """

  # 1. TRAVELER_ID에 대한 USER_ID 열 추가
  concat_merged_df["USER_ID"] = concat_merged_df.groupby("TRAVELER_ID").ngroup() + 1

  # 2. VISIT_AREA_NM에 대한 ITEM_ID 열 추가
  concat_merged_df["ITEM_ID"] = concat_merged_df.groupby("VISIT_AREA_NM").ngroup() + 1

  # 2.1 모든 area 정보 all_area_info.csv로 저장
  all_area_info_dir = dataset_dir + "/all-area-info"
  all_area_info_df = concat_merged_df[["ITEM_ID", "VISIT_AREA_NM", "X_COORD", "Y_COORD"]]
  all_area_info_df = all_area_info_df.sort_values(by="ITEM_ID").copy()
  all_area_info_df.drop_duplicates(inplace=True)
  # all_area_info_df["ADDRESS"] = all_area_info_df.apply(lambda row: convert_coordinate_to_address(row["Y_COORD"], row["X_COORD"]), axis=1)
  # print(all_area_info_df["ADDRESS"].unique())

  all_area_info_df.to_csv(all_area_info_dir + "/all-area-info-address.csv", index=False)

  # 3. DGSTFN -> RATING으로 열 이름 변경
  concat_merged_df.rename(columns={"DGSTFN": "RATING"}, inplace=True)

  # 4. 3개의 열["USER_ID", "ITEM_ID", "RATING"]의 Data Type Integer로 변경
  concat_merged_df = concat_merged_df.astype({"USER_ID": int, "ITEM_ID": int, "RATING": int}) # 모든 열 int로 변경

  # 5. rating_matrix 생성: USER_ID, ITEM_ID, RATING열 추출
  rating_matrix = concat_merged_df[["USER_ID", "ITEM_ID", "RATING"]]

  # 6. ITEM_ID 열을 기준으로 정렬
  rating_matrix = rating_matrix.sort_values(by="ITEM_ID")

  # 7. 섞인 인덱스 재조정
  rating_matrix.reset_index(drop=True, inplace=True)
  rating_matrix.index += 1

  return concat_merged_df, rating_matrix

In [34]:
# method to preprocess the travel_log dataset
def preprocess_data(concat_travel_df, concat_area_df, concat_traveller_df):
  """
    Input
    concat_dataset method로 만들어진 데이터프레임을 입력함
    - concat_travel_df
    - concat_area_df
    - concat_traveller_df

    Output
    rating_matrix는 train_valid의 데이터가 결합된 행렬
    - rating_matrix: 3개의 열['user_id', 'item_id', 'rating']로 구성된 Dataframe이다.
    - rating_matrix.csv 파일
  """
  print("[preprocess_data() 실행]")

  # 1. 필수 관광지 카테고리 필터링
  # 각각의 카테고리 설명 => 1: 자연관광지 2: 역사/유적/종교 ... 10: 상점 --> 렌터카, 편의점 등
  essential_area_types = [1, 2, 3, 4, 5, 6, 7, 8, 13] # 11 맛집 제거
  concat_area_df = concat_area_df[concat_area_df["VISIT_AREA_TYPE_CD"].isin(essential_area_types)]
  print("- 필수 관광지 카테고리 필터링 완료")

  # 2. 문화지구 관광지 전처리
  essential_culture_areas = ["갤러리", "관", "광장", "당", "문화", "뮤지엄", "미술", "센터", "전시", "아트", "촌", "홀"]
  concat_area_df = concat_area_df[(concat_area_df['VISIT_AREA_TYPE_CD'] != 3) | concat_area_df['VISIT_AREA_NM'].str.contains('|'.join(essential_culture_areas))]
  print("- 문화지구 관광지 필터링 완료")

  # 3. 상업지구 관광지 전처리
  essential_business_areas = ["거리", "골목", "광장", "길", "숲", "시장"]
  concat_area_df = concat_area_df[(concat_area_df['VISIT_AREA_TYPE_CD'] != 4) | concat_area_df['VISIT_AREA_NM'].str.contains('|'.join(essential_business_areas))]
  print("- 상업지구 관광지 필터링 완료")

  # 4. VISIT_AREA_NM 전처리
  concat_area_df["VISIT_AREA_NM"] = concat_area_df["VISIT_AREA_NM"].str.strip()
  print("- VISIT_AREA_NM.strip() 완료")

  # 5. 사용할 column 추출
  concat_area_df = concat_area_df[["VISIT_AREA_ID", "VISIT_AREA_NM", "TRAVEL_ID", "DGSTFN", "X_COORD", "Y_COORD"]]
  concat_travel_df = concat_travel_df[["TRAVEL_ID", "TRAVELER_ID"]]
  print("- 사용할 column 추출 완료")

  # 6. null 제거
  concat_area_df = concat_area_df.dropna().copy()
  concat_travel_df = concat_travel_df.dropna().copy()
  print("- NaN Drop 완료")

  # 7. 데이터 프레임 결합 : travel + traveler + area
  concat_merged_df = pd.merge(concat_travel_df, concat_area_df, on="TRAVEL_ID", how="inner") # (43897, 7)
  concat_merged_df = pd.merge(concat_merged_df, concat_traveller_df["TRAVELER_ID"], on="TRAVELER_ID", how="inner") # (39391, 7)
  print("- 데이터 프레임 결합 완료")

  # 8. rating_matrix 생성
  concat_merged_df, rating_matrix = create_rating_matrix(concat_merged_df)
  print("- Rating Matrix 생성 완료")

  # 8. rating_matrix 저장
  rating_matrix.to_csv(dataset_dir + "/cold/raw-rating-matrix.csv", index=False, header=False)
  print("- rating_matrix.csv 저장 완료")

  print("[preprocess_data() 종료]\n\n")

  return concat_merged_df, rating_matrix

In [10]:
# method to create user latent vector
def create_latent_vectors(rating_matrix):
  """
    Input
    - rating_matrix
      rating_matrix는 3개의 열["USER_ID", "ITEM_ID", "RATING"]로 구성된 Dataframe이다.

    Output
    - user_factors, item_factors (latent vectors)
    - WRMF_U.txt (latent vector)
    - WRMF_V.txt (latent vector)
  """

  # 1. 희소 행렬(sparse matrix) 생성
  data = np.array(rating_matrix["RATING"])
  row_indices = np.array(rating_matrix['USER_ID']) # 인덱스로 치환된 ID 필요
  col_indices = np.array(rating_matrix['ITEM_ID'])
  ratings_csr_matrix = csr_matrix((data, (row_indices, col_indices)))

  # 2. bm25_weight 적용 --> implicit Document 참고
  ratings_csr_matrix = bm25_weight(ratings_csr_matrix, K1=100, B=0.8)

  # get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
  traveler_area = ratings_csr_matrix.T.tocsr()

  # 3. ALS 모델 적용 --> WMF?
  model = AlternatingLeastSquares(factors=200, regularization=0.05, alpha=2.0)
  model.fit(ratings_csr_matrix)

  # 4. Latent Vector 생성
  user_factors = model.user_factors
  item_factors = model.item_factors
  # print(user_factors.shape)
  # print(item_factors.shape)
  # print(type(model.user_factors))
  # print(type(model.item_factors))

  # 5. Latent Vector 저장
  current_time = datetime.datetime.now()
  formatted_time = current_time.strftime("%Y-%m-%d %H:%M") # 현재 시간 변수

  WRMF_U_dir = dataset_dir + f"/trained/cold/WRMF_U_{formatted_time}.txt"
  WRMF_I_dir = dataset_dir + f"/trained/cold/WRMF_V_{formatted_time}.txt"

  np.savetxt(WRMF_U_dir, user_factors, fmt="%.7f")
  np.savetxt(WRMF_I_dir, item_factors, fmt="%.7f")

  return user_factors, item_factors

In [11]:
# method to create the user feature vector
def create_user_feature_vector(concat_merged_df, concat_traveller_df):
  """
    Input
    - concat_merged_df: preprocess_data()로부터 나온 concat_merged_df

    Output
    - libsvm_data: user_feature_vector화된 array
    - user_feature_0based_{formatted_time}.txt 파일
  """

  # 1. 활용할 열 추출
  concat_traveller_df = concat_traveller_df[["TRAVELER_ID", "GENDER", "AGE_GRP", "TRAVEL_STYL_1", "TRAVEL_STYL_5", "TRAVEL_STYL_6"]]

  # 2. 성별 값 수치로 변경
  concat_traveller_df.loc[concat_traveller_df["GENDER"] == "남", "GENDER"] = 0
  concat_traveller_df.loc[concat_traveller_df["GENDER"] == "여", "GENDER"] = 1

  # 3. GENDER 컬럼의 데이터 유형을 int로 변경
  concat_traveller_df = concat_traveller_df.astype({"GENDER": int})

  # 4. merged & traveler merged & 열 추출
  merged_df = pd.merge(concat_traveller_df, concat_merged_df["TRAVELER_ID"].drop_duplicates(), on="TRAVELER_ID", how="right")
  merged_df = merged_df[["GENDER", "AGE_GRP", "TRAVEL_STYL_1", "TRAVEL_STYL_5", "TRAVEL_STYL_6"]]

  # 4. 데이터프레임을 LibSVM 형식으로 변환
  # 4.1 변수 선언
  libsvm_data = []
  column_to_num = {
    "GENDER": 1,
    "AGE_GRP": 2,
    "TRAVEL_STYL_1": 3,
    "TRAVEL_STYL_5": 4,
    "TRAVEL_STYL_6": 5
  }

  # 4.2 데이터 프레임을 각 열마다 libsvm 형식 변환
  for index, row in merged_df.iterrows():
    label = index  # 데이터프레임의 인덱스를 레이블로 사용
    features = row.to_dict()
    libsvm_line = f"{label+1} "

    for feature_index, feature_value in features.items():
      libsvm_line += f"{column_to_num[feature_index]}:{feature_value} "
    libsvm_data.append(libsvm_line.strip())

  # 4.3 libsvm 파일 저장
  current_time = datetime.datetime.now()
  formatted_time = current_time.strftime("%Y-%m-%d %H:%M") # 현재 시간 변수

  feature_output_dir = dataset_dir + f"/user_features_0based_{formatted_time}.txt" # 저장할 경로

  # 결과 출력
  with open(feature_output_dir, "w") as output_file:
    output_file.write("0\n")
    for line in libsvm_data:
      output_file.write(f"{line}\n")

  return libsvm_data

In [44]:
# method to create csv file for Training
def create_train_test_test_id_csv_file(rating_matrix):
  """
    Input
    - rating_matrix: preprocess_data()로부터 나온 rating_matrix

    Output
    8:2 비율로 나눠진 훈련, 테스트용 rating_matrix + test 아이템 아이디
    - train_rating_matrix, test_rating_matrix, test_item_ids
    - train.csv, test.csv, test_item_ids.csv

  """
  # 1. ITEM_ID 리스트 만들기
  item_id_df = rating_matrix["ITEM_ID"]
  item_id_list = item_id_df.unique()

  # 2. ITEM_ID (8:2 = Train:Test) 비율로 나누기
  np.random.shuffle(item_id_list) # 배열 랜덤 섞기

  split_ratio = 0.8 # 8:2 비율
  split_index = int(len(item_id_list) * split_ratio)

  train_id_list = item_id_list[:split_index]
  test_id_list =  item_id_list[split_index:]

  # 3. 비율 대로 각각 train/test rating_matrix 생성 + test_item_ids
  train_rating_matrix = rating_matrix[rating_matrix["ITEM_ID"].isin(train_id_list)] # 12630
  test_rating_matrix = rating_matrix[rating_matrix["ITEM_ID"].isin(test_id_list)] # 2781
  test_item_ids = item_id_df[item_id_df.isin(test_id_list)] # 2781

  # 4. train.csv, test.csv, test_ids.csv 파일 저장
  train_rating_matrix.to_csv(dataset_dir + "/cold/train.csv", index=False, header=False)
  test_rating_matrix.to_csv(dataset_dir + "/cold/test.csv", index=False, header=False)
  test_item_ids.to_csv(dataset_dir + "/cold/test_item_ids.csv", index=False, header=False)
  rating_matrix.iloc[0:0].to_csv(dataset_dir + "/cold/zero.csv", index=False, header=False)
  rating_matrix.to_csv(dataset_dir + "/cold/all.csv", index=False, header=False)
  item_id_df.to_csv(dataset_dir + "/cold/all_item_ids.csv", index=False, header=False)

  return train_rating_matrix, test_rating_matrix, test_item_ids

# 데이터 전처리 코드

In [39]:
# 1. 모든 여행 로그 데이터 concat
concat_travel_df, concat_area_df, concat_traveller_df = concat_dataset(sudo_dir, east_dir, west_dir, jeju_dir)

print(concat_travel_df.shape) # 3600 x 4
print(concat_traveller_df.shape) # 3600 x 4
print(concat_area_df.shape)

(14400, 10)
(14400, 36)
(80721, 24)


<ipython-input-33-85771dca0d04>:49: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  concat_area_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone Design 2023 2nd/DropoutNet/Dataset/all.csv')


In [40]:
# 2. 데이터 전처리 -> Rating Matrix 생성
concat_merged_df, rating_matrix = preprocess_data(concat_travel_df, concat_area_df, concat_traveller_df)
print("[preprocess_data() 결과]")
print(f'Number of USER_ID: {len(concat_merged_df["TRAVELER_ID"].unique())}') # User: 11686
print(f'Number of ITEM_ID: {len(concat_merged_df["VISIT_AREA_NM"].unique())}') # Item_name: 9716
print(f"Shape of concat_merged_df: {concat_merged_df.shape}")
print(f"Shape of rating_matrix{rating_matrix.shape}")

[preprocess_data() 실행]
- 필수 관광지 카테고리 필터링 완료
- 문화지구 관광지 필터링 완료
- 상업지구 관광지 필터링 완료
- VISIT_AREA_NM.strip() 완료
- 사용할 column 추출 완료
- NaN Drop 완료
- 데이터 프레임 결합 완료
- Rating Matrix 생성 완료
- rating_matrix.csv 저장 완료
[preprocess_data() 종료]


[preprocess_data() 결과]
Number of USER_ID: 11686
Number of ITEM_ID: 9716
Shape of concat_merged_df: (39308, 9)
Shape of rating_matrix(39308, 3)


In [41]:
# 3. User, Item의 rating에 대한 latent vector 생성
user_latent_vector, item_latent_vector = create_latent_vectors(rating_matrix)
print(f"user_latent_vector : {user_latent_vector}")
print(f"item_latent_vector : {item_latent_vector}")

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0004563331604003906 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

user_latent_vector : [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.42162684  0.76973855 -0.6469293  ...  0.06300142  0.49892542
  -0.335173  ]
 [-0.69350946  0.7979954  -0.3496633  ...  1.5728766   1.1553632
  -1.1671017 ]
 ...
 [ 0.25970533 -0.09942697 -0.05946957 ...  0.15516937  0.28068736
   0.44063723]
 [ 0.17915466  0.16633144  0.3185367  ... -0.23621795 -0.2598626
  -0.10386725]
 [-0.5640088  -0.68615115  0.12926985 ...  0.4197499   1.2489283
   0.42375708]]
item_latent_vector : [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.01084139 -0.00433716  0.00170851 ... -0.00096981  0.00190629
  -0.00083243]
 [ 0.00487677  0.00871187 -0.00122403 ...  0.00952738  0.00735362
   0.01143648]
 ...
 [ 0.00291967  0.01368745  0.00649374 ...  0.00582616  0.00511319
   0.00677657]
 [ 0.00923777  0.00082504  0.00338828 ... -0.00526908  0.00089957
   0.00371515]
 [ 0.0008047   0.00508538  0.00719513 ...  0.01388939  0.00158607
   0.0032

In [42]:
libsvm_data = create_user_feature_vector(concat_merged_df, concat_traveller_df)
print(len(libsvm_data))

11686


In [45]:
train_rating_matrix, test_rating_matrix, test_item_ids = create_train_test_test_id_csv_file(rating_matrix)

print(train_rating_matrix.shape) # 31782 = 39308 * 0.8
print(test_rating_matrix.shape) # 7526 = 39308 * 0.2
print(test_item_ids.shape) # 7526

(31892, 3)
(7416, 3)
(7416,)
